In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/fundos-imobiliarios/busca-avancada')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a/span').click()
time.sleep(2)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/funds.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [3]:
fundinvest = pd.read_csv('funds.csv', sep=';',  decimal=',', thousands='.', encoding="utf-8" )
#fiis = pd.DataFrame(funds)


fundinvest = fundinvest.loc[:, ['TICKER', 'PRECO', 'GESTAO' , 'DY', 'P/VP', 'PERCENTUAL EM CAIXA', 
                        'N COTISTAS', 'CAGR DIVIDENDOS 3 ANOS', ' CAGR VALOR CORA 3 ANOS', 
                        'LIQUIDEZ MEDIA DIARIA', 'PATRIMONIO', 'VALOR PATRIMONIAL COTA', 
                        ' N COTAS', 'ULTIMO DIVIDENDO']]

fundinvest.fillna(value=0, inplace=True)

fundinvest.rename(columns={'PERCENTUAL EM CAIXA': "PERC_CX" }, inplace=True)
fundinvest.rename(columns={'CAGR DIVIDENDOS 3 ANOS': "DY_CAGR_3ANOS" }, inplace=True)
fundinvest.rename(columns={' CAGR VALOR CORA 3 ANOS': "VLR_CAGR_3ANOS" }, inplace=True)
fundinvest.rename(columns={'LIQUIDEZ MEDIA DIARIA': "LIQ_DIARIA" }, inplace=True)
fundinvest.rename(columns={'VALOR PATRIMONIAL COTA': "VLR_PATR_COTA" }, inplace=True)
fundinvest.rename(columns={'ULTIMO DIVIDENDO': "ULT_DIV" }, inplace=True)

fundinvest.rename(columns={'TICKER': "TICKERS" }, inplace=True)


fundinvest = fundinvest.loc[fundinvest['DY'] > 0].loc[
    fundinvest['DY_CAGR_3ANOS'] >= 0.01].loc[
    fundinvest['VLR_CAGR_3ANOS'] >= 0.01].loc[
    fundinvest['N COTISTAS'] >= 1000].loc[
    fundinvest['LIQ_DIARIA'] >= 100].loc[
    fundinvest['ULT_DIV'] >= 0.1]

len(fundinvest.index)

26

In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = fundinvest['TICKERS']
list_funds = ticker.to_list()

indicators_list_funds = []
for funds in list_funds:

    #Criando url
    url = f"https://statusinvest.com.br/fundos-imobiliarios/{funds}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        NAME = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/h1/small').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[3]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[5]/div/div[1]/strong').text
        TIPO = navegador.find_element(By.XPATH, '//*[@id="fund-section"]/div/div/div[2]/div/div[5]/div/div/div/strong').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="fund-section"]/div/div/div[2]/div/div[6]/div/div/strong').text
        
        

        #Criando dict
        dicionario = {  "funds": funds,
                        "NAME": NAME,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "TIPO": TIPO,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        indicators_list_funds.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()

In [5]:
funds = pd.DataFrame.from_dict(indicators_list_funds)

funds = funds.replace('-', '')
funds = funds.replace('-%', '')
funds['MIN_MES'] = (funds['MIN_MES']).str.lstrip('R$ ')
funds['MAX_MES'] = (funds['MAX_MES']).str.lstrip('R$ ')
funds['MIN_MES'] = (funds['MIN_MES']).str.strip('- ')
funds['MAX_MES'] = (funds['MAX_MES']).str.strip('- ')
funds['VAL_12M'] = (funds['VAL_12M']).str.rstrip('%')
funds['VAL_MES'] = (funds['VAL_MES']).str.rstrip('%')

funds.to_csv('list_funds.csv')

list_funds = pd.read_csv("list_funds.csv", sep=',', decimal=',', thousands='.', encoding="utf-8" )

list_funds.fillna(value=0, inplace=True)
list_funds = list_funds.drop(['Unnamed: 0'],axis=1)
list_funds.rename(columns={'funds': "TICKERS" }, inplace=True)

invest_funds = pd.merge(fundinvest, list_funds, on='TICKERS')

invest_funds['UP/DOWNSIDE'] = (invest_funds['VLR_PATR_COTA'] / invest_funds['PRECO'])-1

invest_funds = invest_funds[['TICKERS', 'NAME',  'SEGMENTO', 'GESTAO', 'N COTISTAS', 
                             'PATRIMONIO', ' N COTAS', 'VLR_PATR_COTA', 'UP/DOWNSIDE', 'PRECO', 
                             'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S', 
                             'VAL_12M', 'ULT_DIV', 'DY', 'DY_CAGR_3ANOS', 'P/VP', 
                             'PERC_CX',  'VLR_CAGR_3ANOS', 'LIQ_DIARIA', ]]

invest_funds = invest_funds.loc[invest_funds['DY'] > 0].loc[
    invest_funds['DY_CAGR_3ANOS'] >= 0.01].loc[
    invest_funds['VLR_CAGR_3ANOS'] >= 0.01].loc[
    invest_funds['N COTISTAS'] >= 1000].loc[
    invest_funds['LIQ_DIARIA'] >= 100].loc[
    invest_funds['ULT_DIV'] >= 0.1].loc[
    invest_funds['UP/DOWNSIDE'] > 0]

# def formatar (valor):
#     return "{:.2}".format(float(valor))

invest_funds['PATRIMONIO'] = invest_funds['PATRIMONIO'] / 1000000
invest_funds[' N COTAS'] = invest_funds[' N COTAS'] / 1000
invest_funds['N COTISTAS'] = invest_funds['N COTISTAS'] / 1000



invest_funds

,TICKERS,NAME,SEGMENTO,GESTAO,N COTISTAS,PATRIMONIO,N COTAS,VLR_PATR_COTA,UP/DOWNSIDE,PRECO,...,MIN_52S,MAX_52S,VAL_12M,ULT_DIV,DY,DY_CAGR_3ANOS,P/VP,PERC_CX,VLR_CAGR_3ANOS,LIQ_DIARIA
0,ARRI11,ÁTRIO REIT RECEBÍVEIS IMOBILIÁRIOS,Títulos e Valores Mobiliários,Ativa,27.581,116.877384,13056.237,8.95,0.005618,8.90,...,8.06,9.43,-1.55,0.10,15.15,3.73,1.00,2.63,0.26,286279.18
1,BMLC11,BM BRASCAN LAJES CORPORATIVAS,Híbrido,Ativa,1.493,110.584605,998.405,110.76,0.054857,105.00,...,98.00,108.99,5.00,0.76,11.53,20.48,0.95,17.27,4.22,33388.82
2,BTCR11,BTG PACTUAL CRÉDITO IMOBILIÁRIO,Títulos e Valores Mobiliários,Ativa,16.934,458.252684,4810.097,95.27,0.082614,88.00,...,85.45,97.79,-5.46,0.90,2.05,32.78,0.92,8.00,1.52,652774.65
3,EURO11,EUROPAR,Logística,Passiva,2.429,124.873196,383.936,325.24,0.332951,244.00,...,218.31,250.00,6.14,1.88,9.16,10.36,0.75,1.54,3.89,36334.46
4,FCFL11,CAMPUS FARIA LIMA,Outros,Ativa,3.589,403.737700,3474.010,116.22,0.004494,115.70,...,98.92,125.97,-3.08,0.81,8.18,8.46,1.00,0.68,0.29,178924.54
5,FIGS11,GENERAL SHOPPING ATIVO E RENDA,Shoppings,Passiva,14.961,217.182009,2850.000,76.20,0.176834,64.75,...,45.57,65.12,12.86,0.45,7.98,23.87,0.85,1.01,1.44,188302.41
9,HGPO11,CSHG PRIME OFFICES FDO INV IMOB,Lajes Corporativas,Passiva,16.426,524.198180,1753.057,299.02,0.093349,273.49,...,238.00,297.00,4.93,1.60,7.05,13.65,0.91,0.58,6.79,706113.97
11,HSML11,HSI MALL FDO INV IMOB,Shoppings,Ativa,163.854,1500.114725,15780.613,95.06,0.011492,93.98,...,74.09,96.92,-0.29,0.74,9.18,21.13,0.99,4.33,0.52,2851309.82
13,JRDM11,SHOPPING JARDIM SUL,Shoppings,Passiva,4.001,224.143181,2522.968,88.84,0.034226,85.90,...,76.56,87.93,0.47,0.40,7.36,28.90,0.97,0.41,4.07,610682.00
14,KNCR11,KINEA RENDIMENTOS IMOBILIÁRIOS,Títulos e Valores Mobiliários,Ativa,272.678,5764.202758,57097.087,100.95,0.013656,99.59,...,93.00,102.40,-1.77,1.20,13.81,49.84,0.99,5.56,3.40,11711816.85
